# **앙상블 학습(ensemble learning)**

정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리짐


1.   **랜덤 포레스트(random forest)**
    
    랜덤 포레스트는 결정 트리를 랜덤하게 만들어 결정 트리(나무)의 숲을 만드는 것이다. 각 결정 트리의 예측을 사용해 최종 예측을 만드는 방식이다.
    *   데이터 생성 방법 : 훈련 데이터에서 랜덤하게 샘플을 추출함(중복 가능)
        
        이러한 방식을 부트스트랩 샘플(bootstrap sample)이라고 한다.부트스르랩 샘플은 훈련 세트의 크기와 같게 만든다
        
        ex) 1,000개의 가방에서 중복하여 1,00개의 샘플을 뽑는다.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
from  sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)
#이전에 결정트리에서는 [0.12... / 0.86.../ 0.007]이런식으로 나왔지만 랜덤포레스트로 인해 특성의 일부를 랜덤하게 선택하였기 때문에 당도의 중요도가 감소하고
#나머지 특성이 증가하였다.
#이는 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻어 과대적함을 줄이고 일반화 성능을 높이는데 도움이 된다.

[0.23167441 0.50039841 0.26792718]


In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)
#랜덤 포레스트에서는 훈련 세트에서 중복을 허용하여 부트스트랩 샘플을 만들다.
#이때 부스스트랩 샘플에 포함되지 않고 남은 샘플을 OOB(out of bag)샘플이라고 한다.
#위 코드는 각 결정트리의 OOB 점수를 평균하여 출력한다 -> 교차 검증 대용

0.8934000384837406


# **엑스트라 트리(extra tree)**

엑스트라 트리는 램덤 포레스트와 매우 비슷하게 동작한다. 기본적으로 100개의 결정 트리를 훈련한다. 랜덤 포레스트와 동일하게 결정 트리가 제공하는 대부분의 매개변수를 지원하고 전체 특성 중에 일부 특성을 램덤하게 선택하여 노드를 분할하는 데 사용한다.

랜덤 포레스트와 차이점은 부트스트랩 샘플을 사용하지 않는 것이다.즉, 결정 트리를 만들 때 전체 훈련 세트를 사용한다.

대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할한다.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# **그레디언트 부스팅(gradient boosting)**

깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법이다. 깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있다.

경사 하강법을 사용하여 트리를 앙상블에 추가하는 방식이다.

*   분류 : 로지스틱 손실 함수
*   회귀 : 평균 제곱 오차 함수

그레디언트 부스팅은 결정 트리를 계속 추가하면서 가장 낮은 곳(손실함수의 낮은 곳)을 찾아 이동한다.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [ ]:
gb.fit(train_input,train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


# **히스토그램 기반 그레이디언트 부스팅(Histogram-based Gradient Boosting)**

히스토그램 기반 그레이디언트 부스팅은 정형데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘이다.

히스토그램 기반 그레디언트 부스팅은 먼저 입력 특성을 256개의 구간으로 나눠 노드를 분할할 때 최적으리 분할을 매우 빠르게 찾을 수 있다.
256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용한다 -> 입력에 누락된 특성이 있더라도 따로 전처리 필요 x

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
#이 함수는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지 계산해준다,
#n_repeats는 랜덤하게 섞을 횟수
print(result.importances_mean)
#나온 결과 값은 중요도(importances), 평균(importances_mean), 표준편차(importances_std이다.

[0.08876275 0.23438522 0.08027708]


In [ ]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923



---
사이킷런 말고 히스토그램 기반 그레이디언트 부스팅 알고리즘을 구현한 라이브러리가 존재한다.


*   XGBoost
*   LightGBM



In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


In [ ]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
